##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM API：使用Node.js快速開始聊天


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/palm_docs/chat_node_quickstart"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />檢視 ai.google.dev 上的檔案</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/cn/palm_docs/chat_node_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/palm_docs/chat_node_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始程式碼</a>
  </td>
</table>


## 概述

此快速入門將示範如何使用 PaLM API，它能讓你取得 Google 最新大語言模型，具備特別針對對話為主的應用實例 (例如聊天機器人)，如 PaLM Node.js SDK。


## 取得 API 金鑰

要開始使用，你需要 [取得 API 金鑰](setup)。
將它設定為環境變數：


In [6]:
import os
os.environ["API_KEY"] = "<YOUR API KEY>"

註解：上面使用 `os.environ` 僅因為這是一個基於 Python 的筆記本。從命令列操作的話，你會使用 `export API_KEY=<YOUR API KEY>`。


## 安裝 API 客戶端

在新的目錄中，使用 npm 初始化 Node.js 專案並安裝
`google-auth` 函式庫：


In [9]:
%%bash
npm init -y

npm install google-auth-library

Wrote to /content/package.json:

{
  "name": "content",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "@google-ai/generativelanguage": "^1.0.1",
    "google-auth-library": "^9.0.0"
  },
  "devDependencies": {},
  "description": ""
}


+ google-auth-library@9.0.0
updated 1 package and audited 74 packages in 0.905s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



npm WARN content@1.0.0 No description
npm WARN content@1.0.0 No repository field.



接下來可以安裝用戶端函式庫：


In [10]:
%%bash
npm install @google-ai/generativelanguage

+ @google-ai/generativelanguage@1.0.1
updated 1 package and audited 74 packages in 2.274s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



npm WARN content@1.0.0 No description
npm WARN content@1.0.0 No repository field.



## 取得 API 金鑰

請按照 [設定
頁面](setup) 的說明，為你的應用程式建立一個 API 金鑰。在下一步你將需要這個 API 金鑰。

## 產生訊息

建立一個新的檔案 `index.js`，並加入以下程式碼，並透過 `API_KEY` 環境變數提供你的PaLM API 金鑰：


In [11]:
%%writefile index.js

const { DiscussServiceClient } = require("@google-ai/generativelanguage");
const { GoogleAuth } = require("google-auth-library");

const MODEL_NAME = "models/chat-bison-001";
const API_KEY = process.env.API_KEY;

const client = new DiscussServiceClient({
  authClient: new GoogleAuth().fromAPIKey(API_KEY),
});

async function main() {
  const result = await client.generateMessage({
    model: MODEL_NAME, // Required. The model to use to generate the result.
    temperature: 0.5, // Optional. Value `0.0` always uses the highest-probability result.
    candidateCount: 1, // Optional. The number of candidate results to generate.
    prompt: {
      // optional, preamble context to prime responses
      context: "Respond to all questions with a rhyming poem.",
      // Optional. Examples for further fine-tuning of responses.
      examples: [
        {
          input: { content: "What is the capital of California?" },
          output: {
            content:
              `If the capital of California is what you seek,
Sacramento is where you ought to peek.`,
          },
        },
      ],
      // Required. Alternating prompt/response messages.
      messages: [{ content: "How tall is the Eiffel Tower?" }],
    },
  });

  console.log(result[0].candidates[0].content);
}

main();

Overwriting index.js


接著執行指令碼：


In [12]:
%%bash
node index.js

The Eiffel Tower is 324 meters tall,
It's a sight to behold, tall and not small.
It's made of iron and weighs 10,100 tons,
It's a wonder of the world, it's a must-see for all.


### 繼續對談

如果要繼續對談，在原本的提示訊息中，你需要加上
已回傳的候補訊息以及你的下一個提示：


In [25]:
%%writefile index.js
const { DiscussServiceClient } = require("@google-ai/generativelanguage");
const { GoogleAuth } = require("google-auth-library");

const MODEL_NAME = "models/chat-bison-001";
const API_KEY = process.env.API_KEY;

const client = new DiscussServiceClient({
  authClient: new GoogleAuth().fromAPIKey(API_KEY),
});

async function main() {
  let first = "Tell me a one short animal fact."
  let messages = [{ content: first }];

  const result = await client.generateMessage({
    model: MODEL_NAME,
    prompt: { messages },
  });

  console.log("User:\n\n", first, "\n\n")
  console.log("Palm:\n\n", result[0].candidates[0].content, "\n\n");

  let second = "Oh, where do those live?"

  messages.push({ content: result[0].candidates[0].content });
  messages.push({ content: second });

  const secondResult = await client.generateMessage({
    model: MODEL_NAME,
    prompt: { messages },
  });

  console.log("User:\n\n", second, "\n\n")
  console.log("Palm:\n\n", secondResult[0].candidates[0].content, "\n\n");
}

main();

Overwriting index.js


In [26]:
%%bash
node index.js

User:

 Tell me a one short animal fact. 


Palm:

 The world's smallest mammal is the bumblebee bat, which weighs less than a penny. 


User:

 Oh, where do those live? 


Palm:

 The bumblebee bat is found in the rainforests of Thailand, Myanmar, and Laos. It is a small, nocturnal bat that feeds on insects. The bumblebee bat is the smallest mammal in the world, weighing only about 2 grams. It is about the size of a bumblebee, hence its name. The bumblebee bat is a very important part of the rainforest ecosystem. It helps to control insect populations and pollinate plants. 


